In [ ]:
#Innodata QC - Matt Cook, September 2022
##Compares IIIF manifest of target with Innodata metadata (mets), vendor deliverable
import json
import time

#I/O
manifest = open("...json", "r")#IIIF manifest
mets = open("...mets copy", "r")#innodata (mets) output
log = open("...txt", "w")#output log

#timestamp log
secs = time.localtime()
log.write("Log for QC session on " + str(time.asctime(secs) + '\n'))
log.write("\n")
          
#data structures
listDRS = {}
innoDRS = []
missing = []

#parse IIIF json and add DRS IDs to list
data = json.load(manifest)
for item in range(len(data['sequences'][0]['canvases'])):
    url = (str((data['sequences'][0]['canvases'][item]['images'][0]['resource']['@id'])))
    label = (str((data['sequences'][0]['canvases'][item]['label'])))
    label = label.split("(")
    label = label[1]
    label = label.strip(")")
    listDRS[label] = [url]
    drsNum = url.split("/")[5:6]
    drsNum = drsNum[0]
    drsNum = drsNum.strip(";][")
    if drsNum not in listDRS:
        listDRS[label] = [drsNum, url]
print(str((len(listDRS))) + " unique DRS image IDs were located in the target IIIF manifest.")
print("\n")

#parse Innodata mets file and 
for line in mets.readlines():
    if "<file ID" in line:
        innOut = line.split(" ")
        innOut = innOut[7].strip('ID="')
        innOut = innOut.split("_")
        innOut = innOut[1]
        if innOut not in innoDRS:
            innoDRS.append(innOut)
print(str((len(innoDRS))) + " DRS IDs were located in the Innodata mets metadata output.")
print("\n")

#compare mets and IIIF DRS ID lists
for key,value in listDRS.items():
    drs = value[0]
    url = value[1]
    if drs not in innoDRS:
        print("The following DRS images are unnaccounted for in Innodata output: ")
        print(url)
        missing.append([key, url])

#write log to disk
if len(missing) != 0:
    log.write("The following DRS images are unnaccounted for in Innodata output: ")
    log.write('\n')
    for item in missing:
        log.write(str(item))
        log.write('\n')
else:
    log.write("All images accounted for; the Innodata output matches IIIF manifest.")

#end program
manifest.close()
mets.close()
log.close()
print("\n")
print("Have a nice day")